# Read data

In [5]:
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
path = Path("dataset_clean/")
data = pd.DataFrame({"Observation":[], "Action":[], "Reward":[], "Rank":[]})

for file_path in path.iterdir():
    if file_path.is_file():
        df = pd.read_pickle(file_path)
        data = pd.concat([data, df], ignore_index=True)
        print(f'{file_path} loaded')
    break

In [ ]:
data.head()

# Init model

In [7]:
import torch
from torch import nn
import sys
sys.path.append('videoAutoencoder')

from videoAE import VideoAE

In [ ]:
tmp = data.head(10)
tmp = tmp.Observation
tmp[2] = np.squeeze(tmp[2], axis=0) 

In [ ]:
for idx, video in enumerate(tmp):
    print(video.shape)

In [ ]:
for idx, video in enumerate(tmp):
    tmp[idx] = video[:2992]
    castdata = tmp[idx].astype(np.float32)
    tmp[idx] = castdata[np.newaxis, :, :, :]

In [ ]:
for idx, video in enumerate(tmp):
    print(video.shape)

In [12]:
from torch.utils.data import DataLoader

dataloader = DataLoader(tmp, batch_size=10, shuffle=True)

In [ ]:
batch_size = 10
n_channels = 1
n_frames = 2992
heigth = 84
width = 84

input_shape = (batch_size, n_channels, n_frames, heigth, width)
latent_dim = 128

model = VideoAE(input_shape, latent_dim)

# Define your loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    for data in dataloader:
        inputs = data  # Adjust as needed based on your data loading logic
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')